# Season 17 Election Notes

>**FORBIDDEN KNOWLEDGE WARNING**
>
>This document contains some information that is not currently present on the main site and may be considered spoilers.
>Continue at your own discretion. Some information in this document cannot be shared on the Discord without spoiler tags.

In [1]:
import pandas
%matplotlib inline
from blaseball_mike.models import *
import plotly.express as plot
import plotly.io as _pio
import plotly.subplots as subplot
from IPython.display import display, Markdown

import os
import sys
pdir = os.path.abspath(os.path.join(os.path.dirname(''), os.path.pardir))
sys.path.append(pdir)
from display import *
from blessings import *
sys.path.remove(pdir)

_pio.renderers.default = "notebook_connected"

pies = Team.load_by_name("Philly Pies")

In [2]:
sim = SimulationData.load()
display(Markdown(f"**Last Updated Season {sim.season}, Day {sim.day}**"))

**Last Updated Season 17, Day 103**

---
## Decrees

Decrees are chosen by majority votes across all teams. `3` decrees will pass this season.


All this season's decrees build various new structures that will presumably add gameplay mechanics.

* Forum: Upshelling Events on the Feed will earn you Coins and Prizes.
* Library: Reveal the Ancient Histories of Blaseball.
* Laboratory: Add a Will that lets you build a custom Item and give it to a Player on your Team.
* Smithy: All non-Playoff Teams from this Season build a Smithy in their Ballpark, which occasionally repairs Items.
* Monument: The Top 99 Events of each Season will be Remembered.
* Fairgrounds: All games have a chance of being Prize Matches, in which the winning Team will open an Item Crate.

___
## Wills

Will are selected for each team by raffle. Each team will draw `2` random wills. The bottom 4 teams in the league by
standings will draw `3`. The chosen wills are random, but wills with more votes are more likely to be selected.

### Plunder
**Steal a Player from the Idols board. Send a random Player from your Team back.**

Due to the random nature of the target on our team, this tests the best case (swapping the Best Player on the Idol Board
for a position with our worst player in that position) and the worst case (swapping the Best Player on the Idol Board
for a position with our best player in that position).

**Best Case Swap:**

In [3]:
idols = [x.player for x in Idol.load().values() if x.player.league_team_id != pies]

idols.sort(key=lambda x: x.batting_rating, reverse=True)
best_batting_idol = idols[0]
idols.sort(key=lambda x: x.pitching_rating, reverse=True)
best_pitching_idol = idols[0]

worst_batter = sort_lineup(pies, 1)[0]
display(Markdown(f"Batting: Swap {best_batting_idol.name} for {worst_batter.name}"))

pandas.DataFrame(trade_players(pies, worst_batter, best_batting_idol))

Batting: Swap Beck Whitney for Yusef Puddles

,Pies
batting_change,+11.7319%
pitching_change,+0.0000%
baserunning_change,+9.2840%
defense_change,+3.4670%


In [4]:
worst_pitcher = sort_rotation(pies, 1)[0]
display(Markdown(f"Pitching: Swap {best_pitching_idol.name} for {worst_pitcher.name}"))

pandas.DataFrame(trade_players(pies, worst_pitcher, best_pitching_idol))


Pitching: Swap Pitching Machine for Henry Marshallow

,Pies
batting_change,+0.0000%
pitching_change,+14.4716%
baserunning_change,+0.0000%
defense_change,+0.0000%


**Worst Case Swap:**

In [5]:
best_batter = sort_lineup(pies, 1, order="best")[0]
display(Markdown(f"Batting: Swap {best_batting_idol.name} for {best_batter.name}"))

pandas.DataFrame(trade_players(pies, best_batter, best_batting_idol))

Batting: Swap Beck Whitney for Jaxon Buckley

,Pies
batting_change,+0.6194%
pitching_change,+0.0000%
baserunning_change,+6.4189%
defense_change,+4.1712%


In [6]:
best_pitcher = sort_rotation(pies, 1, order="best")[0]
display(Markdown(f"Pitching: Swap {best_pitching_idol.name} for {best_pitcher.name}"))

pandas.DataFrame(trade_players(pies, best_pitcher, best_pitching_idol))


Pitching: Swap Pitching Machine for Elvis Figueroa

,Pies
batting_change,+0.0000%
pitching_change,+4.3523%
baserunning_change,+0.0000%
defense_change,+0.0000%


### Equivalent Exchange
**Swap a Player from a Team in the opposite Subleague with a Player from your Team. They must be within 2 combined stars of each other.**

See below for the 20 best swap candidates for a particular player, sorted by highest stats in the applicable position.

In [7]:
wild = Subleague.load("aabc11a1-81af-4036-9f18-229c759ca8a9")
all_wild = []
for team in wild.teams.values():
    all_wild.extend(team.lineup + team.rotation)

In [8]:
all_total_stars = get_total_stars(all_wild)
for player in pies.lineup:
    total_stars = get_total_stars(player)[player.id]
    valid_total_stars = {k: v for k, v in all_total_stars.items() if total_stars-2 < v < total_stars+2}
    player_matches = [x for x in all_wild if x.id in valid_total_stars.keys()]
    player_matches.sort(key=lambda x: x.batting_rating, reverse=True)
    display(Markdown(f"####{player.name}"))
    display(get_stars([player] + player_matches[0:20], include_team=True))
    display(set_heatmap(get_batting_stlats([player] + player_matches[0:20]), maxVal=1.5))

####Eduardo Woodman

,,Batting,Pitching,Baserunning,Defense
,Team,,,,
Eduardo Woodman,Pies,4.0,2.0,3.5,3.0
Jacob Haynes,Flowers,5.5,3.5,2.5,2.0
Math Velazquez,Spies,5.0,3.5,4.0,2.5
Comfort Septemberish,Spies,5.0,1.0,5.0,2.5
August Sky,Jazz Hands,4.5,4.5,2.5,3.0
Baby Triumphant,Firefighters,4.5,2.0,2.5,3.0
Bauer Zimmerman,Jazz Hands,4.5,1.5,3.0,2.5
Avila Guzman,Spies,4.5,1.0,5.0,4.0
Reese Clark,Spies,4.5,3.0,2.0,4.0


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Eduardo Woodman,0.833060,1.019391,0.846540,0.650666,0.575519,0.595517,0.574554,0.100000,0.705869
Jacob Haynes,1.126749,1.334891,1.020797,1.141949,1.265377,0.001000,0.440711,0.013502,0.883104
Math Velazquez,0.981884,1.027667,1.067639,0.709586,1.146343,0.395484,0.622834,0.060000,0.682863
Comfort Septemberish,0.978574,1.177996,1.187902,0.375357,0.767285,0.611622,1.105479,0.060000,0.832023
August Sky,0.943651,1.072420,0.913515,0.993340,0.772470,0.268792,0.465689,0.001000,0.460190
Baby Triumphant,0.927529,0.939663,1.109047,0.626394,0.730501,0.001000,0.217618,0.047784,0.847350
Bauer Zimmerman,0.884063,0.963640,0.967942,0.413869,0.969917,0.426705,0.926229,0.100000,0.225597
Avila Guzman,0.877579,0.473915,1.413259,1.078152,1.325349,0.205733,0.758002,0.001000,0.764657
Reese Clark,0.866074,0.897601,0.825922,0.776604,1.028210,0.358106,1.031754,0.060000,0.896331
Felix Garbage,0.859852,0.766287,1.174788,0.656731,0.329195,0.001000,1.036772,0.001000,0.680168


####Beasley Day

,,Batting,Pitching,Baserunning,Defense
,Team,,,,
Beasley Day,Pies,4.5,3.0,3.5,3.0
Jessica Telephone,Lift,5.5,2.5,4.0,3.0
Jacob Haynes,Flowers,5.5,3.5,2.5,2.0
Baby Doyle,Jazz Hands,5.5,2.5,3.0,4.5
Vessalius Sundae,Worms,5.5,3.0,4.0,3.0
Math Velazquez,Spies,5.0,3.5,4.0,2.5
Comfort Septemberish,Spies,5.0,1.0,5.0,2.5
August Sky,Jazz Hands,4.5,4.5,2.5,3.0
Baby Triumphant,Firefighters,4.5,2.0,2.5,3.0


...

,,Batting,Pitching,Baserunning,Defense
,Team,,,,
Yusef Puddles,Pies,2.0,2.5,3.0,3.5
Rat Mason,Tacos,5.0,1.5,2.5,1.5
Summers Preston,Dale,5.0,1.5,1.5,2.0
Baby Triumphant,Firefighters,4.5,2.0,2.5,3.0
Bauer Zimmerman,Jazz Hands,4.5,1.5,3.0,2.5
Randy Castillo,Tigers,4.5,1.5,2.0,2.0
Felix Garbage,Tacos,4.5,3.5,2.0,3.0
Denzel Scott,Spies,4.5,2.5,2.5,1.5
Chambers Simmons,Flowers,4.0,2.5,4.0,2.0


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Yusef Puddles,0.429562,1.033698,0.184577,0.675196,0.151908,0.630778,0.110821,0.057271,1.010015
Rat Mason,1.009065,1.290628,0.798484,1.077502,0.906716,0.030115,1.092972,0.001000,0.653243
Summers Preston,0.974990,0.898723,1.244346,1.227772,0.360495,0.001000,0.848852,0.001000,0.783555
Baby Triumphant,0.927529,0.939663,1.109047,0.626394,0.730501,0.001000,0.217618,0.047784,0.847350
Bauer Zimmerman,0.884063,0.963640,0.967942,0.413869,0.969917,0.426705,0.926229,0.100000,0.225597
Randy Castillo,0.868039,1.030142,1.096949,0.686627,0.257003,0.629329,0.804416,0.022455,1.085623
Felix Garbage,0.859852,0.766287,1.174788,0.656731,0.329195,0.001000,1.036772,0.001000,0.680168
Denzel Scott,0.850411,0.963515,0.849616,0.566107,0.627399,0.119743,0.673793,0.007747,0.949707
Chambers Simmons,0.799968,0.735538,0.975124,0.888963,0.822225,0.765255,0.580358,0.001000,0.642499
Fran Beans,0.785526,0.853002,0.978388,0.568602,0.482162,0.685546,0.313539,0.001000,0.881484


####Usurper Violet

,,Batting,Pitching,Baserunning,Defense
,Team,,,,
Usurper Violet,Pies,3.0,3.0,1.5,4.0
Jacob Haynes,Flowers,5.5,3.5,2.5,2.0
Rat Mason,Tacos,5.0,1.5,2.5,1.5
Summers Preston,Dale,5.0,1.5,1.5,2.0
Baby Triumphant,Firefighters,4.5,2.0,2.5,3.0
Bauer Zimmerman,Jazz Hands,4.5,1.5,3.0,2.5
Randy Castillo,Tigers,4.5,1.5,2.0,2.0
Reese Clark,Spies,4.5,3.0,2.0,4.0
Felix Garbage,Tacos,4.5,3.5,2.0,3.0


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Usurper Violet,0.579715,0.894648,0.293503,1.022726,0.869795,0.569699,0.283561,0.100000,0.487176
Jacob Haynes,1.126749,1.334891,1.020797,1.141949,1.265377,0.001000,0.440711,0.013502,0.883104
Rat Mason,1.009065,1.290628,0.798484,1.077502,0.906716,0.030115,1.092972,0.001000,0.653243
Summers Preston,0.974990,0.898723,1.244346,1.227772,0.360495,0.001000,0.848852,0.001000,0.783555
Baby Triumphant,0.927529,0.939663,1.109047,0.626394,0.730501,0.001000,0.217618,0.047784,0.847350
Bauer Zimmerman,0.884063,0.963640,0.967942,0.413869,0.969917,0.426705,0.926229,0.100000,0.225597
Randy Castillo,0.868039,1.030142,1.096949,0.686627,0.257003,0.629329,0.804416,0.022455,1.085623
Reese Clark,0.866074,0.897601,0.825922,0.776604,1.028210,0.358106,1.031754,0.060000,0.896331
Felix Garbage,0.859852,0.766287,1.174788,0.656731,0.329195,0.001000,1.036772,0.001000,0.680168
Denzel Scott,0.850411,0.963515,0.849616,0.566107,0.627399,0.119743,0.673793,0.007747,0.949707


In [9]:
for player in pies.rotation:
    total_stars = get_total_stars(player)[player.id]
    valid_total_stars = {k: v for k, v in all_total_stars.items() if total_stars-2 < v < total_stars+2}
    player_matches = [x for x in all_wild if x.id in valid_total_stars.keys()]
    player_matches.sort(key=lambda x: x.pitching_rating, reverse=True)
    display(Markdown(f"####{player.name}"))
    display(get_stars([player] + player_matches[0:20], include_team=True))
    display(set_heatmap(get_pitching_stlats([player] + player_matches[0:20]), maxVal=1.5))

####Henry Marshallow

,,Batting,Pitching,Baserunning,Defense
,Team,,,,
Henry Marshallow,Pies,3.5,4.0,2.5,4.5
Patchwork Southwick,Worms,4.0,5.5,3.0,4.0
Rafael Davids,Wild Wings,3.5,5.0,2.5,4.0
Xandra Pancakes,Worms,2.5,5.0,4.0,4.5
Sixpack Santiago,Dale,2.0,5.0,4.5,3.0
Liam Snail,Dale,3.0,4.5,3.5,4.0
Brock Forbes,Flowers,2.0,4.5,3.5,4.0
Miguel James,Sunbeams,3.5,4.5,4.0,3.0
Cell Barajas,Wild Wings,3.5,4.5,2.5,3.5


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,0.768310,0.491270,1.171077,1.428524,0.822605,0.808351,0.876106
Patchwork Southwick,1.062257,1.540457,0.697152,1.346364,0.526488,1.388100,0.970742
Rafael Davids,1.024624,0.877302,1.168981,1.132574,1.063992,1.102898,1.225367
Xandra Pancakes,0.983680,1.082650,1.004702,0.855130,0.854402,0.470938,0.725690
Sixpack Santiago,0.969910,0.765921,1.199201,1.531857,0.747720,0.826116,0.559834
Liam Snail,0.927995,1.032431,0.858471,0.775995,1.083139,1.016481,0.996009
Brock Forbes,0.925907,1.051020,1.196748,0.490476,0.580398,0.607876,0.990014
Miguel James,0.897735,1.259409,0.825396,0.491799,0.882421,0.333087,0.715153
Cell Barajas,0.897041,1.000652,1.021738,0.796261,0.660169,0.187258,0.212798
Howell Rocha,0.889888,0.991393,0.917381,0.727428,0.863727,0.538830,0.487715


####Nerd Pacheco

,,Batting,Pitching,Baserunning,Defense
,Team,,,,
Nerd Pacheco,Pies,4.5,4.0,4.5,3.0
Yummy Elliott,Tacos,4.0,5.5,4.5,4.0
Patchwork Southwick,Worms,4.0,5.5,3.0,4.0
Rafael Davids,Wild Wings,3.5,5.0,2.5,4.0
Xandra Pancakes,Worms,2.5,5.0,4.0,4.5
Walton Sports,Jazz Hands,2.5,5.0,4.5,5.5
Sixpack Santiago,Dale,2.0,5.0,4.5,3.0
Engine Eberhardt,Lift,3.5,4.5,5.5,4.0
Yurts Buttercup,Georgias,4.5,4.5,3.5,4.0


...

,,Batting,Pitching,Baserunning,Defense
,Team,,,,
Elvis Figueroa,Pies,4.0,6.0,2.5,5.0
Cory Twelve,Flowers,5.0,6.0,4.5,3.5
Jayden Wright,Sunbeams,4.0,5.5,4.5,4.5
Yummy Elliott,Tacos,4.0,5.5,4.5,4.0
Patchwork Southwick,Worms,4.0,5.5,3.0,4.0
Rigby Friedrich,Georgias,4.0,5.0,4.5,4.5
Rafael Davids,Wild Wings,3.5,5.0,2.5,4.0
Qais Dogwalker,Dale,3.5,5.0,5.0,4.5
Xandra Pancakes,Worms,2.5,5.0,4.0,4.5


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Elvis Figueroa,1.173083,1.217065,1.206718,1.134739,0.851240,0.514752,0.772422
Cory Twelve,1.165726,1.219391,1.245387,0.908311,0.777330,1.271451,0.428576
Jayden Wright,1.139956,1.048365,1.177798,1.063803,1.352076,1.104007,0.610450
Yummy Elliott,1.067907,0.933410,1.263936,1.252751,0.746269,1.080218,1.066324
Patchwork Southwick,1.062257,1.540457,0.697152,1.346364,0.526488,1.388100,0.970742
Rigby Friedrich,1.030718,1.215808,1.022046,0.550512,1.213816,0.786244,0.656845
Rafael Davids,1.024624,0.877302,1.168981,1.132574,1.063992,1.102898,1.225367
Qais Dogwalker,1.016129,0.958834,1.026445,1.223849,1.039135,0.739043,0.469360
Xandra Pancakes,0.983680,1.082650,1.004702,0.855130,0.854402,0.470938,0.725690
Walton Sports,0.973727,1.259897,0.798052,0.686300,1.135276,0.722438,1.201971


####Tiana Takahashi

,,Batting,Pitching,Baserunning,Defense
,Team,,,,
Tiana Takahashi,Pies,1.5,4.0,3.0,3.5
Silvia Rugrat,Wild Wings,3.0,5.0,1.5,2.0
Wyatt Mason IV,Tacos,1.0,4.5,2.0,4.0
Howell Rocha,Jazz Hands,3.0,4.5,3.0,3.0
Alexandria Rosales,Spies,3.5,4.5,2.0,2.0
Wyatt Pothos,Jazz Hands,2.5,4.0,3.5,3.0
Montgomery Bullock,Georgias,1.5,4.0,2.0,4.0
Lowe -orbe-,Jazz Hands,2.5,4.0,3.5,4.0
Jacoby Podcast,Worms,3.0,4.0,2.5,4.0


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Tiana Takahashi,0.810165,0.947905,1.114604,0.997307,0.116079,0.633482,1.019964
Silvia Rugrat,0.964018,1.022203,1.171607,0.574934,0.730968,1.145101,1.276643
Wyatt Mason IV,0.909013,0.859159,1.110696,0.885281,0.977619,0.194373,0.879172
Howell Rocha,0.889888,0.991393,0.917381,0.727428,0.863727,0.538830,0.487715
Alexandria Rosales,0.850843,1.047870,0.946711,0.989889,0.210874,0.792020,0.787636
Wyatt Pothos,0.844384,0.701425,1.063023,1.132335,0.691922,1.079506,0.799864
Montgomery Bullock,0.819070,1.278244,0.648692,0.631618,0.431200,1.165453,0.458598
Lowe -orbe-,0.780885,1.156714,0.552445,0.496683,1.460711,0.534085,1.387567
Jacoby Podcast,0.775008,0.655825,1.074474,0.642677,0.932817,1.023811,0.197180
Riley Firewall,0.746472,0.743714,0.942633,0.768359,0.504081,0.601937,0.222965


### Foreshadow
**Swap a Player from your Shadows to your active roster.**

There are 14 shadows players to swap with our 12 existing players. Below is a breakdown comparing the best shadows
players with the worst player in the particular position.

Note that position does not matter for the swap: any shadows player can bat or pitch.


**Batting:**

In [10]:
shadows = pies.bullpen + pies.bench
shadows.sort(key=lambda x: x.batting_rating, reverse=True)
best_batting_shadows = shadows[0:4]
worst_batter = sort_lineup(pies, 1)[0]

display(get_stars([worst_batter] + best_batting_shadows))
display(set_heatmap(get_batting_stlats([worst_batter] + best_batting_shadows), maxVal=1))

,Batting,Pitching,Baserunning,Defense
Yusef Puddles,2.0,2.5,3.0,3.5
Peanut Holloway,6.5,4.5,5.5,6.0
Haruta Byrd,3.5,2.5,2.5,1.5
Javier Lotus,3.5,1.0,3.5,4.0
August Obrien,3.5,1.0,3.5,2.0


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Yusef Puddles,0.429562,1.033698,0.184577,0.675196,0.151908,0.630778,0.110821,0.057271,1.010015
Peanut Holloway,1.286142,0.851898,0.833918,0.708373,0.751382,0.143342,0.558181,0.100000,0.460595
Haruta Byrd,0.724391,0.721433,0.917077,0.853010,0.585921,0.832203,0.169263,0.100000,0.124161
Javier Lotus,0.682621,0.938964,0.386700,0.901328,0.942211,0.205027,0.884526,0.100000,0.815592
August Obrien,0.681669,0.641217,0.927569,0.913891,0.492708,0.923854,0.556849,0.100000,0.463958


**Pitching:**

In [11]:
shadows.sort(key=lambda x: x.pitching_rating, reverse=True)
best_pitching_shadows = shadows[0:4]
worst_pitcher = sort_rotation(pies, 1)[0]

display(get_stars([worst_pitcher] + best_pitching_shadows))
display(set_heatmap(get_pitching_stlats([worst_pitcher] + best_pitching_shadows), maxVal=1))

,Batting,Pitching,Baserunning,Defense
Henry Marshallow,3.5,4.0,2.5,4.5
Peanut Holloway,6.5,4.5,5.5,6.0
Lucy Tokkan,2.0,3.5,1.0,3.0
Doc Anice,3.0,3.5,3.0,4.5
Twofurious Puddles,2.5,3.5,2.0,3.5


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,0.768310,0.491270,1.171077,1.428524,0.822605,0.808351,0.876106
Peanut Holloway,0.898684,0.722434,0.588934,0.919687,0.436348,1.093499,0.888946
Lucy Tokkan,0.741748,0.901966,0.995562,0.263758,0.973020,0.180980,0.321805
Doc Anice,0.734279,1.076739,0.487964,0.959145,0.632001,0.765347,0.305586
Twofurious Puddles,0.694922,0.811110,0.650034,0.807712,0.718615,0.416240,0.725894


### Move
**Move a Player on your Team to the desired location on your Roster.**

Move would move a player from any location on the team (including Shadows) to another position, increasing the number
of players in that position. See below for moving our best hitting pitcher or best pitching hitter into their preferred
position.

**Batting**

In [12]:
bhp = best_hitting_pitcher(pies)
bph = best_pitching_hitter(pies)

display(Markdown(f"Moving {bhp.name} to the Lineup"))
display(pandas.DataFrame(move_player(pies, bhp, "lineup")))

display(Markdown(f"Moving {worst_batter.name} to the Shadows"))
display(pandas.DataFrame(move_player(pies, worst_batter, "bench")))

Moving Nerd Pacheco to the Lineup

,Pies
batting_change,+1.1431%
pitching_change,+2.0267%
baserunning_change,+1.7584%
defense_change,-1.7209%


Moving Yusef Puddles to the Shadows

,Pies
batting_change,+5.3934%
pitching_change,+0.0000%
baserunning_change,+2.0815%
defense_change,+0.0186%


**Pitching**

In [13]:
display(Markdown(f"Moving {bph.name} to the Rotation"))
display(pandas.DataFrame(move_player(pies, bph, "rotation")))

display(Markdown(f"Moving {worst_pitcher.name} to the Shadows"))
display(pandas.DataFrame(move_player(pies, worst_pitcher, "bullpen")))

Moving Jaxon Buckley to the Rotation

,Pies
batting_change,-7.3066%
pitching_change,-0.9549%
baserunning_change,-1.1928%
defense_change,+0.8234%


Moving Henry Marshallow to the Shadows

,Pies
batting_change,+0.0000%
pitching_change,+4.2870%
baserunning_change,+0.0000%
defense_change,+0.0000%


### Transfuse
**Boost a Player on your Team by 15% to 30% in a Stat Category of your choice, but Impair them in a Random other Category.**

The stat that is negatively impacted is random. It is generally better to boost pitchers, as only the pitching stats are
used for that position. See below for how this would affect various team members at the 2 extreme percentages. This will
only affect one player.

**15% Change:**

In [14]:
infuse_lineup_table, _, _ = improve_team_batting_table(pies, 0.15)
display(infuse_lineup_table)
display(set_heatmap(get_batting_stlats(improve_team_batting(pies, 0.15)), maxVal=1.5))

,old_batting_stars,new_batting_stars,change_in_batting_stars
Eduardo Woodman,4.165300,4.885272,0.719972
Beasley Day,4.274532,4.998553,0.724021
Huber Frumple,4.287244,5.039095,0.751851
Ruslan Greatness,4.755482,5.439372,0.683890
Jaxon Buckley,6.592812,7.226397,0.633585
Lang Richardson,3.162214,3.907698,0.745485
Yusef Puddles,2.147811,3.068746,0.920935
Usurper Violet,2.898577,3.704516,0.805939


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Eduardo Woodman,0.977054,1.169391,0.996540,0.800666,0.725519,0.445517,0.724554,0.010000,0.855869
Beasley Day,0.999711,1.195160,1.054890,0.522953,0.697377,0.061371,0.883784,0.010000,1.159991
Huber Frumple,1.007819,0.657458,1.460733,1.082690,1.371501,0.076988,0.835091,0.010000,1.263159
Ruslan Greatness,1.087874,0.772589,1.295689,1.719998,1.620457,0.010000,1.461486,0.010000,0.833552
Jaxon Buckley,1.445279,1.717382,1.192101,1.896709,2.080045,0.010000,2.136793,0.010000,1.543504
Lang Richardson,0.781540,0.962722,0.550621,0.772843,1.091821,0.055246,0.648400,0.010000,0.701015
Yusef Puddles,0.613749,1.183698,0.334577,0.825196,0.301908,0.480778,0.260821,0.010000,1.160015
Usurper Violet,0.740903,1.044648,0.443503,1.172726,1.019795,0.419699,0.433561,0.010000,0.637176


In [15]:
infuse_rotation_table, _, _ = improve_team_pitching_table(pies, 0.15)
display(infuse_rotation_table)
display(set_heatmap(get_pitching_stlats(improve_team_pitching(pies, 0.15)), maxVal=1.5))


,old_pitching_stars,new_pitching_stars,change_in_pitching_stars
Henry Marshallow,3.841550,4.774535,0.932985
Nerd Pacheco,4.180600,5.151709,0.971109
Elvis Figueroa,5.865415,6.788955,0.923540
Tiana Takahashi,4.050827,5.114910,1.064082


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,0.954907,0.641270,1.321077,1.578524,0.972605,0.958351,1.026106
Nerd Pacheco,1.030342,1.237720,1.216351,0.422113,0.758134,1.082212,0.634500
Elvis Figueroa,1.357791,1.367065,1.356718,1.284739,1.001240,0.664752,0.922422
Tiana Takahashi,1.022982,1.097905,1.264604,1.147307,0.266079,0.783482,1.169964


**30% Change:**

In [16]:
infuse_lineup_table, _, _ = improve_team_batting_table(pies, 0.30)
display(infuse_lineup_table)
display(set_heatmap(get_batting_stlats(improve_team_batting(pies, 0.30)), maxVal=1.5))

,old_batting_stars,new_batting_stars,change_in_batting_stars
Eduardo Woodman,4.165300,5.585918,1.420618
Beasley Day,4.274532,5.676802,1.402269
Huber Frumple,4.287244,5.741328,1.454084
Ruslan Greatness,4.755482,6.104166,1.348684
Jaxon Buckley,6.592812,7.852389,1.259577
Lang Richardson,3.162214,4.606233,1.444019
Yusef Puddles,2.147811,3.885398,1.737587
Usurper Violet,2.898577,4.460398,1.561822


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Eduardo Woodman,1.117184,1.319391,1.146540,0.950666,0.875519,0.295517,0.874554,0.010000,1.005869
Beasley Day,1.135360,1.345160,1.204890,0.672953,0.847377,0.010000,1.033784,0.010000,1.309991
Huber Frumple,1.148266,0.807458,1.610733,1.232690,1.521501,0.010000,0.985091,0.010000,1.413159
Ruslan Greatness,1.220833,0.922589,1.445689,1.869998,1.770457,0.010000,1.611486,0.010000,0.983552
Jaxon Buckley,1.570478,1.867382,1.342101,2.046709,2.230045,0.010000,2.286793,0.010000,1.693504
Lang Richardson,0.921247,1.112722,0.700621,0.922843,1.241821,0.010000,0.798400,0.010000,0.851015
Yusef Puddles,0.777080,1.333698,0.484577,0.975196,0.451908,0.330778,0.410821,0.010000,1.310015
Usurper Violet,0.892080,1.194648,0.593503,1.322726,1.169795,0.269699,0.583561,0.010000,0.787176


In [17]:
infuse_rotation_table, _, _ = improve_team_pitching_table(pies, 0.30)
display(infuse_rotation_table)
display(set_heatmap(get_pitching_stlats(improve_team_pitching(pies, 0.30)), maxVal=1.5))


,old_pitching_stars,new_pitching_stars,change_in_pitching_stars
Henry Marshallow,3.841550,5.714489,1.872939
Nerd Pacheco,4.180600,6.110125,1.929525
Elvis Figueroa,5.865415,7.728461,1.863046
Tiana Takahashi,4.050827,6.101567,2.050740


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,1.142898,0.791270,1.471077,1.728524,1.122605,1.108351,1.176106
Nerd Pacheco,1.222025,1.387720,1.366351,0.572113,0.908134,1.232212,0.784500
Elvis Figueroa,1.545692,1.517065,1.506718,1.434739,1.151240,0.814752,1.072422
Tiana Takahashi,1.220313,1.247905,1.414604,1.297307,0.416079,0.933482,1.319964


### Alternate Trust
**Call in an Alternate for a Player on your Team, with a mystery Benefit.**

An Alternate is a player reroll and the Benefit is an unknown modification. Below shows the change in Team Average Stars from rerolling our worst
player, assuming they would be replaced with the Statistically Average player.

In [18]:
worst_pie = sort_overall(pies, 1)[0]

display(Markdown(f"Rerolling {worst_pie.name}"))
if worst_pie in pies.rotation:
    out = replace_player(pies, worst_pie, pitch_star=NEW_PLAYER["pitching"])
else:
    out = replace_player(pies, worst_pie, bat_star=NEW_PLAYER["batting"], run_star=NEW_PLAYER["baserunning"], def_star=NEW_PLAYER["defense"])
pandas.DataFrame(out)

Rerolling Yusef Puddles

,Pies
batting_change,-0.3695%
pitching_change,+0.0000%
baserunning_change,-0.7936%
defense_change,-2.6761%


### Reform
**Re-roll an eligible Permanent Modification for a Player on your Team.**

Below is a list of all players with modifications that could be rerolled. It would be better to reroll modifications
with negative effects.

In [19]:
mod_players = [x for x in pies.lineup + pies.rotation if len(x.perm_attr) > 0]
mod_list = pandas.DataFrame()
for player in mod_players:
    data = pandas.Series({"mods": [x.title for x in player.perm_attr if x.id not in UNREFORMED_MODS]}, name=player.name)
    if len(data["mods"]) == 0:
        continue
    mod_list = mod_list.append(data)
mod_list

,mods
Beasley Day,[Minimalist]
Ruslan Greatness,[Friend of Crows]
Jaxon Buckley,[Fire Eater]
Henry Marshallow,[Maximalist]


### Enrich
**Pick a modification for your Team next season.**

This will gives the option of choosing between
* The Fourth Strike: Those with the Fourth Strike will get an extra strike in each at bat.
* High Pressure: This Team will play 25% better in Flooding weather when runners are on base.
* Life of the Party: This team gets 10% more from their Party Time stat boosts.
* Home Field Advantage: Teams with Home Field Advantage will start each home game with one run.
* Affinity for Crows: Players with Affinity for Crows will hit and pitch 50% better during Birds weather.
* Traveling: Traveling teams will play 5% better in Away games.

___
## Blessings

Blessings are selected by raffle across all teams. Each blessing will be awarded to one random team, where teams with
more votes are more likely to be selected.

### Buddies
**Two Adjacent Players in your Team's Lineup will gain +15% Moxie.**

Moxie is the attribute most associated with getting walks (plate discipline).
See below for how this would change the attributes of each player.

In [20]:
new_team = []
for player in pies.lineup:
    new_team.append(player.simulated_copy(buffs={"moxie": 0.15}))
display(get_stars(new_team))
display(set_heatmap(get_batting_stlats(new_team), maxVal=1))

,Batting,Pitching,Baserunning,Defense
Eduardo Woodman,4.0,2.0,3.5,3.0
Beasley Day,4.5,3.0,3.5,3.0
Huber Frumple,4.5,2.0,4.0,3.0
Ruslan Greatness,5.0,4.0,5.0,5.0
Jaxon Buckley,6.5,4.0,4.0,3.5
Lang Richardson,3.0,0.5,3.5,3.5
Yusef Puddles,2.5,2.5,3.0,3.5
Usurper Violet,3.0,3.0,1.5,4.0


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Eduardo Woodman,0.847658,1.019391,0.846540,0.650666,0.725519,0.595517,0.574554,0.100000,0.705869
Beasley Day,0.870577,1.045160,0.904890,0.372953,0.697377,0.211371,0.733784,0.001000,1.009991
Huber Frumple,0.864930,0.507458,1.310733,0.932690,1.371501,0.226988,0.685091,0.041371,1.113159
Ruslan Greatness,0.958051,0.622589,1.145689,1.569998,1.620457,0.001000,1.311486,0.001000,0.683552
Jaxon Buckley,1.325985,1.567382,1.042101,1.746709,2.080045,0.001000,1.986793,0.001000,1.393504
Lang Richardson,0.639492,0.812722,0.400621,0.622843,1.091821,0.205246,0.498400,0.001000,0.551015
Yusef Puddles,0.452270,1.033698,0.184577,0.675196,0.301908,0.630778,0.110821,0.057271,1.010015
Usurper Violet,0.586674,0.894648,0.293503,1.022726,1.019795,0.569699,0.283561,0.100000,0.487176


### Power Chaarge
**Chaarge! Give your Team AA Blood.**

This would make all our players have AA Blood, and give the team a blood-specific Modification. The effects of this
modification are unknown.

### Xenomorph
**Give your Team Acidic Blood.**

This would make all our players have Acidic Blood, and give the team a blood-specific Modification. The effects of this
modification are unknown.

### On Deck
**Call the Alternate for all Players in your Shadows.**

This would randomize all players in our shadows.

### Supply Run
**All Players on your Team with an available Item slot (empty or broken) will open a Generic Jersey Crate.**

**All Players on your Team with an available Item slot (empty or broken) will open a Generic Ring Crate.**

**All Players on your Team with an available Item slot (empty or broken) will open a Generic Necklace Crate.**

**All Players on your Team with an available Item slot (empty or broken) will open a Generic Sunglasses Crate.**

For reference, these items boost the following:


| Item Type  | Attributes | Stat |
| ---------- | ---------- | ---- |
| Jersey     | +Musclitude | Batting |
| Ring       | +Ruthlessness | Pitching |
| Necklace   | +Moxie | Batting |
| Sunglasses | +Cinnamon | Vibes |

This would provide Generic Jerseys, Rings, Necklaces, and Sunglasses to the following players:

In [21]:
free_items = [x for x in pies.lineup + pies.rotation if (x.evolution + 1) - len([x for x in x.items if x.health > 0]) > 0]
get_stars(free_items)

,Batting,Pitching,Baserunning,Defense
Huber Frumple,4.5,2.0,4.0,3.0
Ruslan Greatness,5.0,4.0,5.0,5.0
Jaxon Buckley,6.5,4.0,4.0,3.5
Yusef Puddles,2.0,2.5,3.0,3.5
Henry Marshallow,3.5,4.0,2.5,4.5
Nerd Pacheco,4.5,4.0,4.5,3.0
Elvis Figueroa,4.0,6.0,2.5,5.0


### Handcrafted Drop
**A Random Hitter on your Team with an available Item slot (empty or broken) will open a Handcrafted Jersey Crate.**

**A Random Pitcher on your Team with an available Item slot (empty or broken) will open a Handcrafted Ring Crate.**

**A Random Hitter on your Team with an available Item slot (empty or broken) will open a Handcrafted Necklace Crate.**

**A Random Player on your Team with an available Item slot (empty or broken) will open a Handcrafted Sunglasses Crate.**

For reference, these items boost the following:

| Item Type  | Attributes | Stat |
| ---------- | ---------- | ---- |
| Jersey     | +Musclitude | Batting |
| Ring       | +Ruthlessness | Pitching |
| Necklace   | +Moxie | Batting |
| Sunglasses | +Cinnamon | Vibes |

This would provide Special Jerseys, Rings, Necklaces, and Sunglasses to one of the following players:

**Jersey or Necklace**

In [22]:
free_items_lineup = [x for x in pies.lineup if (x.evolution + 1) - len([x for x in x.items if x.health > 0]) > 0]
get_stars(free_items_lineup)

,Batting,Pitching,Baserunning,Defense
Huber Frumple,4.5,2.0,4.0,3.0
Ruslan Greatness,5.0,4.0,5.0,5.0
Jaxon Buckley,6.5,4.0,4.0,3.5
Yusef Puddles,2.0,2.5,3.0,3.5


**Ring**

In [23]:
free_items_rotation= [x for x in pies.rotation if (x.evolution + 1) - len([x for x in x.items if x.health > 0]) > 0]
get_stars(free_items_rotation)

,Batting,Pitching,Baserunning,Defense
Henry Marshallow,3.5,4.0,2.5,4.5
Nerd Pacheco,4.5,4.0,4.5,3.0
Elvis Figueroa,4.0,6.0,2.5,5.0


**Sunglasses**

In [24]:
get_stars(free_items)

,Batting,Pitching,Baserunning,Defense
Huber Frumple,4.5,2.0,4.0,3.0
Ruslan Greatness,5.0,4.0,5.0,5.0
Jaxon Buckley,6.5,4.0,4.0,3.5
Yusef Puddles,2.0,2.5,3.0,3.5
Henry Marshallow,3.5,4.0,2.5,4.5
Nerd Pacheco,4.5,4.0,4.5,3.0
Elvis Figueroa,4.0,6.0,2.5,5.0


### Crate Drop
**A Random Hitter on your Team with an available Item slot (empty or broken) will open a Dexterity Crate.**

**A Random Player on your Team with an available Item slot (empty or broken) will open a Slime Crate.**

**A Random Hitter on your Team with an available Item slot (empty or broken) will open a Blood Crate.**

**A Random Pitcher on your Team with an available Item slot (empty or broken) will open an Intelligence Crate.**

For reference, these items boost the following:

| Item Type    | Attributes | Stat |
| ------------ | ---------- | ---- |
| Dexterity    | +Thwackability, +Omniscience, +Continuation | +Batting, +Defense, +Baserunning |
| Slimy        | -Laserlikeness, +Watchfulness, +Omniscience, +Tenaciousness  | -Baserunning, +Defense |
| Blood        | +Divinity, +Martyrdom, -Basethirst | +Batting, -Baserunning |
| Intelligence | +Ruthlessness, +Anticapitalism, +Shakespearianism | +Pitching, +Defense |

This will open various crates for a random player on our team.


**Dexterity or Blood**

In [25]:
free_items_lineup = [x for x in pies.lineup if (x.evolution + 1) - len([x for x in x.items if x.health > 0]) > 0]
get_stars(free_items_lineup)

,Batting,Pitching,Baserunning,Defense
Huber Frumple,4.5,2.0,4.0,3.0
Ruslan Greatness,5.0,4.0,5.0,5.0
Jaxon Buckley,6.5,4.0,4.0,3.5
Yusef Puddles,2.0,2.5,3.0,3.5


**Intelligence**

In [26]:
free_items_rotation= [x for x in pies.rotation if (x.evolution + 1) - len([x for x in x.items if x.health > 0]) > 0]
get_stars(free_items_rotation)

,Batting,Pitching,Baserunning,Defense
Henry Marshallow,3.5,4.0,2.5,4.5
Nerd Pacheco,4.5,4.0,4.5,3.0
Elvis Figueroa,4.0,6.0,2.5,5.0


**Slime**

In [27]:
get_stars(free_items)

,Batting,Pitching,Baserunning,Defense
Huber Frumple,4.5,2.0,4.0,3.0
Ruslan Greatness,5.0,4.0,5.0,5.0
Jaxon Buckley,6.5,4.0,4.0,3.5
Yusef Puddles,2.0,2.5,3.0,3.5
Henry Marshallow,3.5,4.0,2.5,4.5
Nerd Pacheco,4.5,4.0,4.5,3.0
Elvis Figueroa,4.0,6.0,2.5,5.0


### In a Pinch
**Promote the Best Baserunner in your Team's Shadows to a random position in the Lineup.**

See below for new player and the change in team stats.

In [28]:
new_lineup = pies.bench + pies.bullpen
new_lineup.sort(key=lambda x: x.baserunning_rating, reverse=True)
best_shadows_runner = new_lineup[0]

display(Markdown(f"Moving {best_shadows_runner.name} to the Lineup"))
display(get_stars([best_shadows_runner] + pies.lineup))
display(set_heatmap(get_baserunning_stlats([best_shadows_runner] + pies.lineup), maxVal=1))
pandas.DataFrame(move_player(pies, best_shadows_runner, "lineup"))

Moving Peanut Holloway to the Lineup

,Batting,Pitching,Baserunning,Defense
Peanut Holloway,6.5,4.5,5.5,6.0
Eduardo Woodman,4.0,2.0,3.5,3.0
Beasley Day,4.5,3.0,3.5,3.0
Huber Frumple,4.5,2.0,4.0,3.0
Ruslan Greatness,5.0,4.0,5.0,5.0
Jaxon Buckley,6.5,4.0,4.0,3.5
Lang Richardson,3.0,0.5,3.5,3.5
Yusef Puddles,2.0,2.5,3.0,3.5
Usurper Violet,3.0,3.0,1.5,4.0


,Baserunning Rating,Laserlikeness,Continuation,Base Thirst,Indulgence,Ground Friction
Peanut Holloway,1.065175,0.618806,0.362588,0.481880,0.848308,1.047406
Eduardo Woodman,0.700068,0.617191,1.040482,0.526095,0.707616,0.815097
Beasley Day,0.725277,0.619583,0.669022,1.189892,0.592646,0.934967
Huber Frumple,0.808039,0.766723,0.710497,0.853913,0.929982,0.793744
Ruslan Greatness,1.032786,1.375092,0.602818,0.693571,1.108719,0.605824
Jaxon Buckley,0.792695,0.789885,1.292203,0.606722,0.613522,0.662351
Lang Richardson,0.716308,0.742498,0.421602,0.845647,0.446945,0.988950
Yusef Puddles,0.563489,0.607851,0.230396,0.996456,0.455636,0.371812
Usurper Violet,0.334915,0.829727,0.996871,0.222526,0.013191,0.015431


,Pies
batting_change,+5.3227%
pitching_change,+0.0000%
baserunning_change,+3.9553%
defense_change,+5.2367%


### Patchwork
**Promote the Best Defender in your Team's Shadows to a random position in the Lineup.**

See below for new player and the change in team stats.

In [29]:
new_lineup.sort(key=lambda x: x.defense_rating, reverse=True)
best_shadows_defender = new_lineup[0]

display(Markdown(f"Moving {best_shadows_defender.name} to the Lineup"))
display(get_stars([best_shadows_defender] + pies.lineup))
display(set_heatmap(get_defense_stlats([best_shadows_defender] + pies.lineup), maxVal=1))
pandas.DataFrame(move_player(pies, best_shadows_defender, "lineup"))

Moving Peanut Holloway to the Lineup

,Batting,Pitching,Baserunning,Defense
Peanut Holloway,6.5,4.5,5.5,6.0
Eduardo Woodman,4.0,2.0,3.5,3.0
Beasley Day,4.5,3.0,3.5,3.0
Huber Frumple,4.5,2.0,4.0,3.0
Ruslan Greatness,5.0,4.0,5.0,5.0
Jaxon Buckley,6.5,4.0,4.0,3.5
Lang Richardson,3.0,0.5,3.5,3.5
Yusef Puddles,2.0,2.5,3.0,3.5
Usurper Violet,3.0,3.0,1.5,4.0


,Defense Rating,Omniscience,Tenaciousness,Watchfulness,Anticapitalism,Chasiness
Peanut Holloway,1.186694,0.418685,0.495287,1.054413,0.901115,0.821125
Eduardo Woodman,0.619466,0.339614,0.324310,0.814677,0.914311,0.920885
Beasley Day,0.609757,0.355386,0.398364,1.304772,0.457722,0.593574
Huber Frumple,0.580774,0.472205,0.267753,0.465153,1.016524,0.577598
Ruslan Greatness,1.012064,1.170016,0.894059,0.903316,0.848210,1.344688
Jaxon Buckley,0.657751,0.636443,0.470729,0.438125,0.927871,0.415402
Lang Richardson,0.748934,1.043616,0.940524,0.374060,0.390675,0.394329
Yusef Puddles,0.714086,0.489394,0.972955,0.277902,0.609092,0.898316
Usurper Violet,0.780281,1.064183,0.740791,0.533876,0.390291,0.646039


,Pies
batting_change,+5.3227%
pitching_change,+0.0000%
baserunning_change,+3.9553%
defense_change,+5.2367%


### Alternator
**Recruit a Random Player in the League with the Alternate Modification to your Team.**

See below for list of all active players with Alternate. This will not remove an existing player from our team.

In [30]:
league = League.load()
divisions = league.subleagues
all_teams = []
for division in divisions.values():
    all_teams.extend(list(division.teams.values()))

In [31]:
siphons = []
for team in all_teams:
    siphons.extend([x for x in team.lineup + team.rotation if "ALTERNATE" in x._perm_attr_ids])
get_stars(siphons, include_team=True)

,,Batting,Pitching,Baserunning,Defense
,Team,,,,
Elwin McGhee,Lift,4.0,2.0,4.5,3.0
Elijah Valenzuela,Jazz Hands,0.5,2.5,2.0,2.0
Campos Arias,Jazz Hands,3.0,3.0,2.5,3.0
Kathy Mathews,Jazz Hands,3.0,2.5,3.0,3.0
Socks Maybe,Firefighters,1.5,0.0,2.5,3.0
Justice Spoon,Firefighters,3.5,0.0,2.0,2.0
Yong Wright,Wild Wings,3.0,0.5,2.5,2.5
Ronan Combs,Wild Wings,2.5,2.0,2.0,2.5
Burke Gonzales,Wild Wings,4.5,6.5,4.5,4.5


### Strange Attractor
**Recruit a Random Player in the League with the Attractor Modification to your Team.**

See below for list of all active players with Attractor. This will not remove an existing player from our team.

Notes:
* This lists the False Stars, they will play using their player attributes instead.

In [32]:
siphons = []
for team in all_teams:
    siphons.extend([x for x in team.lineup + team.rotation if "ATTRACTOR" in x._perm_attr_ids])
display(get_stars(siphons, include_team=True))

,,Batting,Pitching,Baserunning,Defense
,Team,,,,
Brisket Friendo,Sunbeams,4.0,1.0,4.0,4.0
Chorby Short,Millennials,5.5,7.5,7.0,5.5
Oscar Dollie,Shoe Thieves,5.0,6.0,5.0,5.5


### Last Not Least
**The Last Three Players in your Team's Lineup will gain 10% Thwackability.**

Thwackability is the attribute most associated with making contact.
See below for how this would change the attributes of each player.

In [33]:
new_team = pies.lineup[0:-3]
for player in pies.lineup[-3:]:
    new_team.append(player.simulated_copy(buffs={"thwackability": 0.1}))
display(get_stars(new_team))
display(set_heatmap(get_batting_stlats(new_team), maxVal=1))

,Batting,Pitching,Baserunning,Defense
Eduardo Woodman,4.0,2.0,3.5,3.0
Beasley Day,4.5,3.0,3.5,3.0
Huber Frumple,4.5,2.0,4.0,3.0
Ruslan Greatness,5.0,4.0,5.0,5.0
Jaxon Buckley,6.5,4.0,4.0,3.5
Lang Richardson,3.5,0.5,3.5,3.5
Yusef Puddles,2.0,2.5,3.0,3.5
Usurper Violet,3.0,3.0,1.5,4.0


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Eduardo Woodman,0.833060,1.019391,0.846540,0.650666,0.575519,0.595517,0.574554,0.100000,0.705869
Beasley Day,0.854906,1.045160,0.904890,0.372953,0.547377,0.211371,0.733784,0.001000,1.009991
Huber Frumple,0.857449,0.507458,1.310733,0.932690,1.221501,0.226988,0.685091,0.041371,1.113159
Ruslan Greatness,0.951096,0.622589,1.145689,1.569998,1.470457,0.001000,1.311486,0.001000,0.683552
Jaxon Buckley,1.318562,1.567382,1.042101,1.746709,1.930045,0.001000,1.986793,0.001000,1.393504
Lang Richardson,0.658658,0.912722,0.400621,0.622843,0.941821,0.205246,0.498400,0.001000,0.551015
Yusef Puddles,0.443672,1.133698,0.184577,0.675196,0.151908,0.630778,0.110821,0.057271,1.010015
Usurper Violet,0.601618,0.994648,0.293503,1.022726,0.869795,0.569699,0.283561,0.100000,0.487176


### Set the Table
**Move the Best Overall Hitter in your Team's Lineup or Rotation to the #3 spot in your Team's Lineup.**

See below for the new lineup order.

In [34]:
new_lineup = pies.lineup.copy()
new_lineup.sort(key=lambda x: x.batting_rating, reverse=True)
best_divinity = new_lineup[0]

new_lineup = pies.lineup.copy()
new_lineup.insert(2, new_lineup.pop(new_lineup.index(best_divinity)))
get_stars(new_lineup)

,Batting,Pitching,Baserunning,Defense
Eduardo Woodman,4.0,2.0,3.5,3.0
Beasley Day,4.5,3.0,3.5,3.0
Jaxon Buckley,6.5,4.0,4.0,3.5
Huber Frumple,4.5,2.0,4.0,3.0
Ruslan Greatness,5.0,4.0,5.0,5.0
Lang Richardson,3.0,0.5,3.5,3.5
Yusef Puddles,2.0,2.5,3.0,3.5
Usurper Violet,3.0,3.0,1.5,4.0


### Grounded
**A random player in your Team's Rotation will gain 40% Overpowerment but lose 20% Unthwackability.**

Overpowerment is the attribute most associated with reducing big hits (Homers, Triples).
Unthwackability is the attribute most associated with reducing hits in general.
See below for how this would change the attributes of each player.

In [35]:
new_team = []
for player in pies.rotation:
    new_team.append(player.simulated_copy(buffs={"overpowerment": 0.4, "unthwackability": -0.2}))
display(get_stars(new_team))
display(set_heatmap(get_pitching_stlats(new_team), maxVal=1))

,Batting,Pitching,Baserunning,Defense
Henry Marshallow,3.5,3.0,2.5,4.5
Nerd Pacheco,4.5,4.5,4.5,3.0
Elvis Figueroa,4.0,5.5,2.5,5.0
Tiana Takahashi,1.5,4.0,3.0,3.5


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,0.613912,0.291270,1.171077,1.828524,0.822605,0.808351,0.876106
Nerd Pacheco,0.865071,0.887720,1.066351,0.672113,0.608134,0.932212,0.484500
Elvis Figueroa,1.122062,1.017065,1.206718,1.534739,0.851240,0.514752,0.772422
Tiana Takahashi,0.756979,0.747905,1.114604,1.397307,0.116079,0.633482,1.019964


### Untouchable
**A random player in your Team's Rotation will gain 40% Unthwackability but lose 20% Overpowerment.**

Overpowerment is the attribute most associated with reducing big hits (Homers, Triples).
Unthwackability is the attribute most associated with reducing hits in general.
See below for how this would change the attributes of each player.

In [36]:
new_team = []
for player in pies.rotation:
    new_team.append(player.simulated_copy(buffs={"overpowerment": -0.2, "unthwackability": 0.4}))
display(get_stars(new_team))
display(set_heatmap(get_pitching_stlats(new_team), maxVal=1))

,Batting,Pitching,Baserunning,Defense
Henry Marshallow,3.5,5.0,2.5,4.5
Nerd Pacheco,4.5,4.0,4.5,3.0
Elvis Figueroa,4.0,6.5,2.5,5.0
Tiana Takahashi,1.5,4.5,3.0,3.5


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,1.011709,0.891270,1.171077,1.228524,0.822605,0.808351,0.876106
Nerd Pacheco,0.801236,1.487720,1.066351,0.072113,0.608134,0.932212,0.484500
Elvis Figueroa,1.313423,1.617065,1.206718,0.934739,0.851240,0.514752,0.772422
Tiana Takahashi,0.934201,1.347905,1.114604,0.797307,0.116079,0.633482,1.019964


---
## Appendix
* [Description of Attributes](https://www.blaseball.wiki/w/Player_Attributes)
* [Stlat Viewer](https://slavfox.space/abslve/?foreboding-kaleidoscope#PHIL)
* [Historical Player Graphs](http://yoori.space/hloroscopes/)
